In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import sys, os
import torch

sys.path.append(os.path.abspath('../'))


from src.task_vectors_adapted import TaskVectorAdapted
from src.eval import eval_single_dataset
from src.args import parse_arguments 

In [ ]:
#Config
datasets = ['MNIST', 'cars']
model = 'ViT-L-14'

sys.argv = [
    'notebook',  # script name
    '--data-location', '../datasets',
    '--model', f'../{model}',
    '--save', f'../checkpoints/{model}',
    '--batch-size', '128'
]
args = parse_arguments()


pretrained_checkpoint = f'../checkpoints/{model}/zeroshot.pt'
finetuned_checkpoints = [f'../checkpoints/{model}/{dataset}/finetuned.pt' for dataset in datasets]

In [5]:
operations = ['subtract', 'add', 'multiply', 'divide']


In [6]:
results = {}
for dataset in datasets:
    print(f"Testing dataset: {dataset}")

    finetuned_checkpoint = f'../checkpoints/{model}/{dataset}/finetuned.pt'
    for operation in operations:
        task_vector = TaskVectorAdapted(
            pretrained_checkpoint=pretrained_checkpoint,
            finetuned_checkpoint=finetuned_checkpoint,
            operation=operation
        )
        print(f"Testing operation: {operation}")
        encoder_test = task_vector.apply_to(pretrained_checkpoint, scaling_coef=0.5)
        metrics = eval_single_dataset(encoder_test, dataset, args)
        results[(dataset, operation)] = metrics

        del task_vector
        del encoder_test

Testing dataset: MNIST
Testing operation: subtract


/Users/giovanniattina/miniconda3/envs/task-vectors/lib/python3.10/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:36<?, ?it/s]


Done evaluating on MNIST. Accuracy: 100.00%
Testing operation: add
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


...

  0%|          | 0/79 [00:33<?, ?it/s]


Done evaluating on MNIST. Accuracy: 9.38%
Testing operation: divide
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:34<?, ?it/s]


Done evaluating on MNIST. Accuracy: 7.81%
Testing dataset: Cars
Testing operation: subtract
Classification head for ../ViT-L-14 on Cars exists at ../checkpoints/ViT-L-14/head_Cars.pt
Loading classification head from ../checkpoints/ViT-L-14/head_Cars.pt


HTTPError: HTTP Error 404: Not Found

In [ ]:
task_vectors = {
    operation: TaskVectorAdapted(
        pretrained_checkpoint=pretrained_checkpoint,
        finetuned_checkpoint=finetuned_checkpoint,
        operation=operation
    )
    for operation in operations
}

In [20]:
results = {}
for operation, task_vector in task_vectors.items():
    print(f"Testing operation: {operation}")
    encoder_test = task_vector.apply_to(pretrained_checkpoint, scaling_coef=0.5)
    metrics = eval_single_dataset(encoder_test, dataset, args)
    results[operation] = metrics

    

Testing operation: subtract


/Users/giovanniattina/miniconda3/envs/task-vectors/lib/python3.10/site-packages/torch/utils/data/dataloader.py:563: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:34<?, ?it/s]


Done evaluating on MNIST. Accuracy: 100.00%
Testing operation: add
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


...

Done evaluating on MNIST. Accuracy: 12.50%
Testing operation: multiply
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:37<?, ?it/s]


Done evaluating on MNIST. Accuracy: 9.38%
Testing operation: divide
Classification head for ../ViT-L-14 on MNIST exists at ../checkpoints/ViT-L-14/head_MNIST.pt
Loading classification head from ../checkpoints/ViT-L-14/head_MNIST.pt


  0%|          | 0/79 [00:36<?, ?it/s]

Done evaluating on MNIST. Accuracy: 7.81%


In [21]:
results

{'subtract': {'top1': 1.0},
 'add': {'top1': 0.125},
 'multiply': {'top1': 0.09375},
 'divide': {'top1': 0.078125}}

In [22]:
import pandas as pd

In [24]:
pd.DataFrame.from_dict(results, orient='index')

,top1
add,0.125000
divide,0.078125
multiply,0.093750
subtract,1.000000
